In [1]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
import os
import pandas as pd
import geopandas as gpd

In [2]:
df = pd.read_csv('../data/final/gbd_IschemicHeartDisease_DeathsIncidence.csv')

In [3]:
df

,measure_name,location_name,cause_name,year,Value,upper,lower
0,Deaths,Greece,Ischemic heart disease,1990,198.963673,206.809839,187.732342
1,Deaths,Lesotho,Ischemic heart disease,1990,38.577482,47.023134,29.860005
2,Deaths,Djibouti,Ischemic heart disease,1990,19.784015,25.606502,15.357142
3,Deaths,Global,Ischemic heart disease,1990,106.468267,110.197494,101.034486
4,Deaths,Chile,Ischemic heart disease,1990,82.872330,85.753086,78.615036
...,...,...,...,...,...,...,...
12355,Incidence,Monaco,Ischemic heart disease,2015,414.483066,480.150553,353.668307
12356,Incidence,Monaco,Ischemic heart disease,2016,415.230774,481.654755,356.698254
12357,Incidence,Monaco,Ischemic heart disease,2017,417.529883,483.928186,358.608463
12358,Incidence,Monaco,Ischemic heart disease,2018,421.057297,487.301826,362.010349


In [4]:
death_df = df[(df.measure_name == "Deaths")]

In [5]:
import sys
sys.path.append('..')
from src.plotting import custom_cmap

from tueplots import bundles
from tueplots.constants.color import rgb

In [77]:
# Load the world map data using geopandas
# plt.rcParams.update(bundles.icml2022(column="full", ncols=2, nrows=1))

def plot_world_map(filename, data, country_col, measured_col, label, year):
    
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

    # Merge your data with the geopandas DataFrame using country names
    merged = world.merge(data[data.year==year], how='left', left_on='name', right_on=country_col)

    # Fill NaN values (countries with no deaths) with 0
    merged[measured_col] = merged[measured_col].fillna(0)
    
    # Plotting the map
    # fig, ax = plt.subplots(1, 1, figsize=(15, 10))
    ax = merged.plot(column=measured_col, cmap='Reds', linewidth=0.8, edgecolor='0.8', legend=True,
                legend_kwds={'label': label, 'orientation': "horizontal"}
               )
    ax.set_title(f'{label} by countries in year {year}')

    ax.axis('off')

    chart = ax.get_figure()
    chart.savefig(filename, dpi=600)
    plt.close()
    # return ax

In [79]:
filenames = []
for year in range(1990, 2020):
    filename = f'year_{year}.png'
    plot_world_map(filename, death_df, country_col='location_name', measured_col='Value',  label="Death Rate", year=year)
    filenames.append(filename)

# Create gif
with imageio.get_writer('world_map_CVDs_by_year.gif', mode='I', duration=0.3) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Remove files
for filename in filenames:
    os.remove(filename)